In [1]:
import tabula

pdf_file = "PDF/R24-00BX-1.pdf"
#pdf_file = "PDF/d_neg_v6.pdf"
dfs = tabula.read_pdf(pdf_file, pages="all", stream=True, encoding='cp1252')
tabula.environment_info()
print(len(dfs))

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


Python version:
    3.13.3 (tags/v3.13.3:6280bb5, Apr  8 2025, 14:47:33) [MSC v.1943 64 bit (AMD64)]
Java version:
    openjdk version "21.0.1" 2023-10-17 LTS
OpenJDK Runtime Environment Microsoft-8526870 (build 21.0.1+12-LTS)
OpenJDK 64-Bit Server VM Microsoft-8526870 (build 21.0.1+12-LTS, mixed mode, sharing)
tabula-py version: 2.10.0
platform: Windows-11-10.0.26100-SP0
uname:
    uname_result(system='Windows', node='DESKTOP-4A7A8NJ', release='11', version='10.0.26100', machine='AMD64')
linux_distribution: ('', '', '')
mac_ver: ('', ('', '', ''), '')
2


In [2]:
print(len(dfs))

2


In [3]:
dfs[len(dfs)-1]

HGVS description: NM_001320.6(CSNK2B):c.256C>T p.(Arg86Cys)  \
0    Genomic coordinates: Chr6(GRCh38):g.31668619C>T            

             Classification: Likely pathogenic  
0  Zygosity: Heterozygous Inheritance: de novo

Based on https://build.fhir.org/ig/HL7/genomics-reporting/

I think we can build a variant http://hl7.org/fhir/uv/genomics-reporting/StructureDefinition/variant

In [53]:
from fhir.resources.observation import Observation,ObservationComponent
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.coding import Coding

df = dfs[len(dfs)-1]
df['Gene'] = df['Gene'].astype(str)
df['HGVS Description'] = df['HGVS Description'].astype(str)
df.dtypes

Gene                       object
Zygosity                   object
Inheritance                object
HGVS Description           object
Location: GRCh38 (hg38)    object
*Classification            object
dtype: object

In [55]:


for i in range(0, len(df)):
    if df.loc[i,'Gene'] is not None and df.loc[i,'Gene'] != 'nan':
        print('\n',df.loc[i,'Gene'],'\n')
        observationBase = {
            "status" : "final",
            "category":[
                {
                    "coding" : [
                        {
                            "system" : "http://terminology.hl7.org/CodeSystem/observation-category",
                            "code" : "laboratory"
                        }
                    ]
                },
                {
                    "coding" : [
                        {
                            "system" : "http://terminology.hl7.org/CodeSystem/v2-0074",
                            "code" : "GE"
                        }
                    ]
                }
            ],
            "code" : {
                "coding" : [
                    {
                        "system" : "http://loinc.org",
                        "code" : "69548-6",
                        "display" : "Genetic variant assessment"
                    }
                ]
            },
            "valueCodeableConcept" : {
                "coding" : [
                    {
                        "system" : "http://loinc.org",
                        "code" : "LA9633-4",
                        "display" : "Present"
                    }
                ]
            },
            "subject" : {
                "reference" : "urn:uuid:d6faafcf-db64-4c11-9da8-25f36774c1bd",
                "identifier" : {
                    "system" : "https://fhir.nhs.uk/Id/nhs-number",
                    "value" : "9449305552"
                },
                "display" : "Octavia CHISLETT"
            },
            "method" : {
                "coding" : [
                    {
                        "system" : "http://loinc.org",
                        "code" : "LA26398-0",
                        "display" : "Sequencing"
                    }
                ]
            },
            "component" : []
        }
        obs = Observation(**observationBase)
        genename = "NK"
        if df.loc[i,'Gene'] == 'BRCA1':
            genename = "HGNC:1100"
        if df.loc[i,'Gene'] == 'BRCA2':
            genename = "HGNC:1101"
        if df.loc[i,'Gene'] == 'PALB2':
            genename = "HGNC:26144"
        geneData = {
            "code" : {
                "coding" : [
                    {
                        "system" : "http://loinc.org",
                        "code" : "48018-6"
                    }
                ]
            },
            "valueCodeableConcept" : {
                "coding" : [
                    {
                        "system" : "http://www.genenames.org",
                        "code" : genename,
                        "display" : df.loc[i,'Gene']
                    }
                ]
            }
        }
        gene = ObservationComponent(**geneData)
        obs.component.append(gene)

        if df.loc[i,'Zygosity'] == 'Heterozygous':
            alleData = {
                "code" : {
                    "coding" : [
                        {
                            "system" : "http://loinc.org",
                            "code" : "53034-5",
                            "display" : "Allelic state"
                        }
                    ]
                },
                "valueCodeableConcept" : {
                    "coding" : [
                        {
                            "system" : "http://loinc.org",
                            "code" : "LA6706-1",
                            "display" : "Heterozygous"
                        }
                    ]
                }
            }
            alle = ObservationComponent(**alleData)
            obs.component.append(alle)
        if df.loc[i,'HGVS Description'] is not None and df.loc[i,'HGVS Description'] != 'nan':
            code = df.loc[i,'Gene'] + ":" + df.loc[i,'HGVS Description']
            hgvsData = {
                "code" : {
                    "coding" : [
                        {
                            "system" : "http://loinc.org",
                            "code" : "48004-6",
                            "display": "DNA change"
                        }
                    ]
                },
                "valueCodeableConcept" : {
                    "coding" : [
                        {
                            "system" : "http://varnomen.hgvs.org",
                            "code" : code
                        }
                    ]
                }
            }
            hgvs = ObservationComponent(**hgvsData)
            obs.component.append(hgvs)

        json_str = obs.model_dump_json()
        print(json_str)


 BRCA1 

{"resourceType":"Observation","status":"final","category":[{"coding":[{"system":"http://terminology.hl7.org/CodeSystem/observation-category","code":"laboratory"}]},{"coding":[{"system":"http://terminology.hl7.org/CodeSystem/v2-0074","code":"GE"}]}],"code":{"coding":[{"system":"http://loinc.org","code":"69548-6","display":"Genetic variant assessment"}]},"subject":{"reference":"urn:uuid:d6faafcf-db64-4c11-9da8-25f36774c1bd","identifier":{"system":"https://fhir.nhs.uk/Id/nhs-number","value":"9449305552"},"display":"Octavia CHISLETT"},"valueCodeableConcept":{"coding":[{"system":"http://loinc.org","code":"LA9633-4","display":"Present"}]},"method":{"coding":[{"system":"http://loinc.org","code":"LA26398-0","display":"Sequencing"}]},"component":[{"code":{"coding":[{"system":"http://loinc.org","code":"48018-6"}]},"valueCodeableConcept":{"coding":[{"system":"http://www.genenames.org","code":"HGNC:1100","display":"BRCA1"}]}},{"code":{"coding":[{"system":"http://loinc.org","code":"53034-